# Session 4: Damage Analyses on Lifelines 

You will learn about damage analyses on Electric Power Facilities, Roadways, Water Facilities and Buried Pipelines in this session.

## 4.1 Hazard - Tsunami
We will use Tsunami hazard for damage analyses on Electric Power Facilities and Roadway. 

EPF damage analysis supports various hazards including earthquake, tsunami, tornado and hurricane. Roadway damage analysis supports earthquake and tsunami. 

In [ ]:
# import modules
from pyincore import IncoreClient, DataService, HazardService, FragilityService, Dataset, MappingSet
import pandas as pd

In [ ]:
from pyincore_viz.geoutil import GeoUtil as geoviz
from pyincore_viz.plotutil import PlotUtil as plotviz
from pyincore_viz.globals import PACKAGE_VERSION as vizversion

In [ ]:
# connect to in-core services 
client = IncoreClient()
print("pyIncore-viz version detected: " + vizversion)

data_service = DataService(client)
hazard_service = HazardService(client)
fragility_service = FragilityService(client)

###  Dataset based probabilistic tsunami - 500 yr

We will use a probabilistic Tsunami (500 years recurrence interval).

In [ ]:
hazard_type = "tsunami"
hazard_id = "5df90e07b9219cd00ce971e7"
demand_type = "Hmax"
tsu_json = HazardService.get_tsunami_hazard_metadata(hazard_service, hazard_id)

for tsu_dataset in tsu_json['hazardDatasets']:
    if tsu_dataset["demandType"].lower() == demand_type.lower():
        geoviz.plot_raster_dataset(tsu_dataset['datasetId'], client)  # Visualize the Tsunami raster

## 4.2. Electric power facilities (EPF) damage
This analysis computes electric power facility damage based on a particular hazard. In this secion, you will perform Electric Power Facility damage anlayis with a probablistic tsuanmi.

The process for computing the structural damage is similar to other parts of the built environment. First, a fragility is obtained based on the hazard type and attributes of the electric power facility. Based on the fragility, the hazard intensity at the location of the electric power facility is computed. Using this information, the probability of exceeding each limit state is computed, along with the probability of damage. For the case of an earthquake hazard, soil information can be used to modify the damage probabilities to include damage due to liquefaction.

The outputs of this analysis are CSV file with probabilities of damage and JSON file with information about hazard and fragilities.

### Visualize input EPF dataset with poles and substations

In [ ]:
epf_id = "5d263f08b9219cf93c056c68"     # electric power poles and substation
epf_dataset = Dataset.from_data_service(epf_id, data_service)

epf_df = epf_dataset.get_dataframe_from_shapefile()
print(epf_df.head(5))
print(epf_df.groupby('utilfcltyc').size())  
# EDC2: Electric Distribution Circuits
# ESSL2: Electric Sub Station (Low Voltage)

geoviz.plot_map(epf_dataset, column=None, category=False)

### Configure and run EPF Damage analysis

In [ ]:
from pyincore.analyses.epfdamage import EpfDamage

epf_dmg = EpfDamage(client)

In [ ]:
epf_dmg.load_remote_input_dataset("epfs", epf_id)
epf_dmg.set_parameter("hazard_type", hazard_type)
epf_dmg.set_parameter("hazard_id", hazard_id)

fragility_service = FragilityService(client)
mapping_id = "5d31eb7fb9219c6d66398445" # Mapping and the individual fragilities can be viewed on DFR3 Viewer
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
epf_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)
epf_dmg.set_parameter('fragility_key', "Non-Retrofit inundationDepth Fragility ID Code")

epf_dmg.set_parameter("num_cpu", 4)
epf_dmg.set_parameter("result_name", "seaside_tsu_epf_damage")
epf_dmg.run_analysis()  # Run Analysis

### Visualize Damage Results - as dataframe

In [ ]:
# getting results from the analysis
epf_result = epf_dmg.get_output_dataset("result")
epf_dmg_df = epf_result.get_dataframe_from_csv()

# join the output dataset and inventory dataset
epf_dmg_gdf = epf_df[['guid', 'utilfcltyc', 'geometry']].merge(epf_dmg_df, on='guid')
epf_dmg_gdf.head(len(epf_dmg_gdf))

### Visualize Damage Results - on map for a single damage state

In [ ]:
# Probability of an EPF to be in damage state 2 (Moderate)
geoviz.plot_join_map(epf_dataset, epf_result, column="DS_2")

### Show damage states by structure type

In [ ]:
grouped_epf_dmg = epf_dmg_gdf.groupby(by=['utilfcltyc'], as_index=True)\
.agg({'DS_0': 'mean', 'DS_1':'mean', 'DS_2': 'mean', 'DS_3': 'mean', 'DS_4': 'mean', 'guid': 'count'})
grouped_epf_dmg.rename(columns={'guid': 'total_count'}, inplace=True)
grouped_epf_dmg.head()

### Plot Damage state by structure type

In [ ]:
ax = grouped_epf_dmg[["DS_0", "DS_1", "DS_2", "DS_3", "DS_4"]].plot.barh(stacked=True)
ax.set_title("Stacked Bar Chart of Damage State Grouped by Structure Type", fontsize=12)
ax.set_xlabel("complete damage value", fontsize=12)
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

### Show probability distribution for a particular damage state

In [ ]:
ax = epf_dmg_gdf['DS_0'].hist(bins=20, figsize=[10,5])
ax.set_title("no damage distribution", fontsize=12)
ax.set_xlabel("no damage probability", fontsize=12)
ax.set_ylabel("count of units", fontsize=12)

## 4.3. Roadway Damage
This analysis computes roadway damage based on a particular hazard. In this secion, you will perform Road damage anlayis for a probablistic tsunami.

The process for computing is very similar with EPF damage analysis. 

Same as EPF damage analysis, the outputs of this analysis are CSV file with probabilities of damage and JSON file with information about hazard and fragilities.

### Visualize input roadway dataset

In [ ]:
road_dataset_id = "60e5e5cd544e944c3ce37d08"     # electric power poles and substation
road_dataset = Dataset.from_data_service(road_dataset_id, data_service)

road_df = road_dataset.get_dataframe_from_shapefile()
print(road_df.head(5))
print(road_df.groupby('highway').size())

geoviz.plot_map(road_dataset, column=None, category=False)

### Configure and run Roadway Damage analysis with repair rate

In [ ]:
from pyincore.analyses.roaddamage import RoadDamage

road_dmg = RoadDamage(client)

In [ ]:
road_dmg.load_remote_input_dataset("roads", road_dataset_id)
road_dmg.set_parameter("hazard_type", hazard_type)
road_dmg.set_parameter("hazard_id", hazard_id)

fragility_service = FragilityService(client)
mapping_id = "5d274fd8b9219c3c553c71ff" # Mapping and the individual fragilities can be viewed on DFR3 Viewer
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
road_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)
road_dmg.set_parameter('fragility_key', "Non-Retrofit inundationDepth Fragility ID Code")

road_dmg.set_parameter("num_cpu", 4)
road_dmg.set_parameter("result_name", "seaside_road_damage")
road_dmg.run_analysis()  # Run Analysis

### Visualize Damage Results - as dataframe

In [ ]:
# getting results
road_dmg_result = road_dmg.get_output_dataset("result")
road_dmg_df = road_dmg_result.get_dataframe_from_csv()

# joining output dataset and inventory dataset
road_dmg_gdf = road_df[['guid', 'highway', 'geometry']].merge(road_dmg_df, on='guid')
road_dmg_gdf.head(len(road_dmg_gdf))

### Visualize Damage Results - on map for a single damage state

In [ ]:
# Probability of a road to be in damage state 3 (Extensive)
geoviz.plot_join_map(road_dataset, road_dmg_result, column="DS_3")

## 4.4. Hazard - Earthquake
We will use an Earthquake hazard for damage analyses on water facilities and buried pipelines.

The inventory datasets are from Memphis testbed.


### Model based earthquake using Atkinson Boore 1995

In [ ]:
hazard_type = "earthquake"
hazard_id = "5b902cb273c3371e1236b36b"
demand_type = "PGA"
eq_json = HazardService.get_earthquake_hazard_metadata(hazard_service, hazard_id)
geoviz.plot_earthquake(hazard_id, client)

liq_geology_dataset_id =  "5a284f53c7d30d13bc08249c"
liquefaction = True
liq_fragility_key = "pgd"

## 4.5 Water Facility Damage Analysis
This analysis computes water facility damage based on a particular hazard. In this section, you will perform water facility damage analysis for a probabilistic tsunami.

The process for computing is very similar with previous damage analyses. 

Same as EPF damage analysis, the outputs of this analysis are CSV file with probabilities of damage and JSON file with information about hazard and fragilities.

### Visualize input water facilities dataset

In [ ]:
wf_dataset_id = "5a284f2ac7d30d13bc081e52"     # water facilities of shelby county
wf_dataset = Dataset.from_data_service(wf_dataset_id, data_service)

wf_df = wf_dataset.get_dataframe_from_shapefile()
print(wf_df.head(5))
print(wf_df.groupby('utilfcltyc').size())
# PPPL: Large Pumping Plant
# PSTAS: Above Ground Steel Tank

geoviz.plot_map(wf_dataset, column=None, category=False)

### Configure and run Water Facility Damage analysis

In [ ]:
from pyincore.analyses.waterfacilitydamage import WaterFacilityDamage

wf_dmg = WaterFacilityDamage(client)

In [ ]:
wf_dmg.load_remote_input_dataset("water_facilities", wf_dataset_id)

wf_dmg.set_parameter("hazard_type", hazard_type)
wf_dmg.set_parameter("hazard_id", hazard_id)
wf_dmg.set_parameter("use_liquefaction", liquefaction)
wf_dmg.set_parameter("liquefaction_geology_dataset_id", liq_geology_dataset_id)
wf_dmg.set_parameter("liquefaction_fragility_key", liq_fragility_key)

mapping_id = "5b47c383337d4a387669d592" #Potable Water Facility Fragility Mapping for INA - Includes PGD curves
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
wf_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)
wf_dmg.set_parameter("fragility_key", "pga")

wf_dmg.set_parameter("num_cpu", 4)
wf_dmg.set_parameter("result_name", "shelby_wf_dmg")
wf_dmg.run_analysis()

### Visualize Damage Results - as dataframe

In [ ]:
wf_result = wf_dmg.get_output_dataset("result")
wf_dmg_df = wf_result.get_dataframe_from_csv()

wf_dmg_gdf = wf_df[['guid', 'utilfcltyc', 'geometry']].merge(wf_dmg_df, on='guid')
wf_dmg_gdf.head(len(wf_dmg_gdf))

### Visualize Damage Results - on map for a single damage state

In [ ]:
# Probability of a water facility to be in damage state 4 (Complete)
geoviz.plot_join_map(wf_dataset, wf_result, column="DS_4")

## 4.6. Water Pipeline Damage with Repairs
This analysis computes buried pipeline damage based on a particular hazard. In this section, you will perform buried pipeline damage analysis for a probabilistic tsunami.

The process for computing is very similar with previous damage analyses. 

Same as EPF damage analysis, the outputs of this analysis are CSV file with probabilities of repair rate and break rate and JSON file with information about hazard and fragilities.

### Visualize input water pipeline dataset

In [ ]:
wp_dataset_id = "5a284f28c7d30d13bc081d14"     # buried water pipelines of shelby county
wp_dataset = Dataset.from_data_service(wp_dataset_id, data_service)

wp_df = wp_dataset.get_dataframe_from_shapefile()
print(wp_df.head(5))

geoviz.plot_map(wp_dataset, column=None, category=False)

### Configure and run Pipeline Damage analysis with repair rate

In [ ]:
from pyincore.analyses.pipelinedamagerepairrate import PipelineDamageRepairRate, PipelineUtil

pipeline_dmg_w_rr = PipelineDamageRepairRate(client)

In [ ]:
pipeline_dmg_w_rr.load_remote_input_dataset("pipeline", wp_dataset_id)

pipeline_dmg_w_rr.set_parameter("hazard_type", hazard_type)
pipeline_dmg_w_rr.set_parameter("hazard_id", hazard_id)
pipeline_dmg_w_rr.set_parameter("liquefaction_fragility_key", liq_fragility_key)
pipeline_dmg_w_rr.set_parameter("liquefaction_geology_dataset_id",liq_geology_dataset_id)
pipeline_dmg_w_rr.set_parameter("use_liquefaction", liquefaction)

mapping_id = "5b47c227337d4a38464efea8"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
pipeline_dmg_w_rr.set_input_dataset("dfr3_mapping_set", mapping_set)

pipeline_dmg_w_rr.set_parameter("num_cpu", 4)
pipeline_dmg_w_rr.set_parameter("result_name", "pipeline_dmg_w_rr")
pipeline_dmg_w_rr.run_analysis()

### Visualize Damage Results - as dataframe

In [ ]:
wp_result = pipeline_dmg_w_rr.get_output_dataset("result")
wp_dmg_df = wp_result.get_dataframe_from_csv()

wp_dmg_gdf = wp_df[['guid', 'pipelinesc', 'geometry']].merge(wp_dmg_df, on='guid')
wp_dmg_gdf.head(len(wp_dmg_gdf))

### Visualize Damage Results - on map for repairs/km

In [ ]:
# Repairs/km for water pipelines
geoviz.plot_join_map(wp_dataset, wp_result, column="repairspkm")

## Useful Links
- Getting started with pyincore and pyincore-viz: https://tools.in-core.org/doc/incore/getting_started.html

- pyIncore Analyses documentation: https://tools.in-core.org/doc/incore/analyses.html

- Example notebooks: https://tools.in-core.org/doc/incore/notebooks.html

- Data Viewer to browse datasets: https://tools.in-core.org/DataViewer

- Hazard Viewer to browse hazards: https://tools.in-core.org/HazardViewer

- DFR3 viewer to browse Fragility, Repair, Restoration curves: https://tools.in-core.org/DFR3Viewer